# Notebook to play around and develop functions

In [1]:
from tvsclib.strict_system import StrictSystem
from tvsclib.stage import Stage
from tvsclib.system_identification_svd import SystemIdentificationSVD
from tvsclib.toeplitz_operator import ToeplitzOperator
from tvsclib.mixed_system import MixedSystem
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as linalg

In [2]:
matrix = np.arange(0,12).reshape((-1,1))@np.arange(0,12).reshape((1,-1))
dims_in =  np.array([2, 1, 2, 1])*2
dims_out = np.array([1, 2, 1, 2])*2
T = ToeplitzOperator(matrix, dims_in, dims_out)
S = SystemIdentificationSVD(T,epsilon=1e-12)

system = MixedSystem(S)

# Test print function

In [3]:
str(system)

'Mixed system with the parts \nCausal System:\n    State dimensions: [1, 1, 1, 0]\n    Input dimensions: [4, 2, 4, 2]\n    Output dimensions:[2, 4, 2, 4]\n    System is minimal\nAnticausal System:\n    State dimensions: [0, 1, 1, 1]\n    Input dimensions: [4, 2, 4, 2]\n    Output dimensions:[2, 4, 2, 4]\n    System is minimal'

In [4]:
print(system)

Mixed system with the parts 
Causal System:
    State dimensions: [1, 1, 1, 0]
    Input dimensions: [4, 2, 4, 2]
    Output dimensions:[2, 4, 2, 4]
    System is minimal
Anticausal System:
    State dimensions: [0, 1, 1, 1]
    Input dimensions: [4, 2, 4, 2]
    Output dimensions:[2, 4, 2, 4]
    System is minimal


# Test is cannonical

In [60]:
matrix = np.array([
        [5,     4,     6,     1,     4,     2],
        [2,     3,     2,     1,     3,     4],
        [6,     3,     5,     4,     1,     1],
        [3,     5,     5,     5,     3,     4],
        [2,     4,     3,     6,     1,     2],
        [2,     4,     4,     1,     5,     4]
])
matrix = np.vstack((np.hstack((matrix,matrix)),np.hstack((matrix,matrix))))

dims_in =  np.array([2, 1, 2, 1])*2
dims_out = np.array([1, 2, 1, 2])*2
T = ToeplitzOperator(matrix, dims_in, dims_out)
S = SystemIdentificationSVD(T,epsilon=1e-10)

testsys = MixedSystem(S).causal_system
testsys_a = MixedSystem(S).anticausal_system
print(testsys)

Causal System:
    State dimensions: [4, 6, 4, 0]
    Input dimensions: [4, 2, 4, 2]
    Output dimensions:[2, 4, 2, 4]
    System is minimal


In [6]:
testsys.is_balanced(tolerance=1e-14)

False

In [7]:
tolerance = 1e-14
obs_matricies = testsys.observability_matricies()
for i in range(len(obs_matricies)):
    obs_gramian = obs_matricies[i].T@obs_matricies[i]
    d_obs = np.diag(obs_gramian).copy()
    np.fill_diagonal(obs_gramian,0)
    obs_orth = np.all(np.abs(obs_gramian) <tolerance)
    print(np.abs(obs_gramian) <tolerance)
    print(obs_orth)
    print(d_obs)

[]
True
[]
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]
True
[24.10260514  6.32501651  4.25050004  0.99591202]
[[ True  True  True  True  True  True]
 [ True  True  True  True  True  True]
 [ True  True  True  True  True  True]
 [ True  True  True  True  True  True]
 [ True  True  True  True  True  True]
 [ True  True  True  True  True  True]]
True
[20.51056908  6.14672642  4.95566612  1.61074959  0.51606658  0.33888728]
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]
True
[23.46617655  6.26159906  5.17575699  0.58521367]


In [8]:
reach_matricies = testsys.reachability_matricies()
for i in range(len(reach_matricies)):
    reach_gramian =reach_matricies[i]@reach_matricies[i].T
    d_reach = np.diag(reach_gramian).copy()
    np.fill_diagonal(reach_gramian,0)
    reach_orth = np.all(np.abs(reach_gramian) <tolerance)
    print(np.abs(reach_gramian) <tolerance)
    print(reach_orth)
    print(d_reach)

[]
True
[]
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]
True
[24.10260514  6.32501651  4.25050004  0.99591202]
[[ True False  True  True  True False]
 [False  True  True  True  True  True]
 [ True  True  True  True  True  True]
 [ True  True  True  True  True  True]
 [ True  True  True  True  True  True]
 [False  True  True  True  True  True]]
False
[20.51056908  6.14672642  4.95566612  1.61074959  0.51606658  0.33888728]
[[ True  True  True False]
 [ True  True  True False]
 [ True  True  True  True]
 [False False  True  True]]
False
[23.46617655  6.26159906  5.17575699  0.58521367]


In [9]:
n = testsys.stages[2].A_matrix.shape[1]
P = np.eye(6)
P[0,0]=0
P[2,0]=1
P[2,2]=0
P[0,2]=1
testsys_permuted = testsys.copy()
testsys_permuted.stages[2].A_matrix = testsys_permuted.stages[2].A_matrix@P
testsys_permuted.stages[2].C_matrix = testsys_permuted.stages[2].C_matrix@P

testsys_permuted.stages[1].A_matrix = P@testsys_permuted.stages[1].A_matrix
testsys_permuted.stages[1].B_matrix = P@testsys_permuted.stages[1].B_matrix

testsys_permuted.is_balanced(tolerance=1e-13)

True

In [10]:
reach_matricies = testsys_permuted.reachability_matricies()
for i in range(len(reach_matricies)):
    reach_gramian =reach_matricies[i]@reach_matricies[i].T
    d_reach = np.diag(reach_gramian).copy()
    np.fill_diagonal(reach_gramian,0)
    reach_orth = np.all(np.abs(reach_gramian) <tolerance)
    print(np.abs(reach_gramian) <tolerance)
    print(reach_orth)
    print(d_reach)
    print(np.all(d_reach[1:]-d_reach[:-1]<0))

[]
True
[]
True
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]
True
[24.10260514  6.32501651  4.25050004  0.99591202]
True
[[ True  True  True  True  True  True]
 [ True  True False  True  True  True]
 [ True False  True  True  True False]
 [ True  True  True  True  True  True]
 [ True  True  True  True  True  True]
 [ True  True False  True  True  True]]
False
[ 4.95566612  6.14672642 20.51056908  1.61074959  0.51606658  0.33888728]
False
[[ True  True  True False]
 [ True  True  True False]
 [ True  True  True  True]
 [False False  True  True]]
False
[23.46617655  6.26159906  5.17575699  0.58521367]
True


In [11]:
testsys.is_input_normal()

False

In [12]:
testsys.is_output_normal()

False

# Test make minimal

In [91]:
def make_reachable(self,tol = 1e-8):
    """ make_reachable makes the system reachable
            This also includes a possible reduction of the state dimension

        Args:
            tol: (float, optional): used for the rank determination in the SVD
                (Note: this is not nececarraly the tolerance for the overall Reachability matrix)

        TODO: it is unclear if this is also true if the first/final state has dim =0
        """
    if self.causal:
        k = len(self.stages)
        for i in range(k-1):
            U,s,Vt= np.linalg.svd(np.hstack([self.stages[i].A_matrix,self.stages[i].B_matrix]))
            n = np.count_nonzero(s>tol)

            rs = np.sqrt(s[:n])
            Us=U[:,:n]*rs
            sVt=rs.reshape(-1,1)*Vt[:n,:]

            self.stages[i].A_matrix=sVt[:,:self.stages[i].A_matrix.shape[1]]
            self.stages[i].B_matrix=sVt[:,self.stages[i].A_matrix.shape[1]:]
            self.stages[i+1].A_matrix = self.stages[i+1].A_matrix@Us
            self.stages[i+1].C_matrix = self.stages[i+1].C_matrix@Us
    else:
        raise NotImplementedError("Not yet implemented")
        
def make_input_normal(self,tol = 1e-8):
    """ make_reachable makes the system reachable
            This also includes a possible reduction of the state dimension

        Args:
            tol: (float, optional): used for the rank determination in the SVD
                (Note: this is not nececarraly the tolerance for the overall Reachability matrix)

        TODO: it is unclear if this is also true if the first/final state has dim =0
        """
    if self.causal:
        k = len(self.stages)
        for i in range(k-1):
            U,s,Vt= np.linalg.svd(np.hstack([self.stages[i].A_matrix,self.stages[i].B_matrix]))
            n = np.count_nonzero(s>tol)

            
            Us=U[:,:n]*s[:n]
            sVt=Vt[:n,:]

            self.stages[i].A_matrix=sVt[:,:self.stages[i].A_matrix.shape[1]]
            self.stages[i].B_matrix=sVt[:,self.stages[i].A_matrix.shape[1]:]
            self.stages[i+1].A_matrix = self.stages[i+1].A_matrix@Us
            self.stages[i+1].C_matrix = self.stages[i+1].C_matrix@Us
    else:
        raise NotImplementedError("Not yet implemented")
        
def make_input_normal_qr(self,tol = 1e-8):
    """ make_reachable makes the system reachable
            This also includes a possible reduction of the state dimension


        TODO: it is unclear if this is also true if the first/final state has dim =0
        """
    if self.causal:
        k = len(self.stages)
        for i in range(k-1):
            Q, R = np.linalg.qr(np.hstack([self.stages[i].A_matrix,self.stages[i].B_matrix]).T,'reduced')
            #U,s,Vt= np.linalg.svd(np.hstack([self.stages[i].A_matrix,self.stages[i].B_matrix]))
            #n = np.count_nonzero(s>tol)

            #Us=U[:,:n]*s[:n]
            #sVt=Vt[:n,:]
            Q = Q.T
            L = R.T

            self.stages[i].A_matrix=Q[:,:self.stages[i].A_matrix.shape[1]]
            self.stages[i].B_matrix=Q[:,self.stages[i].A_matrix.shape[1]:]
            self.stages[i+1].A_matrix = self.stages[i+1].A_matrix@L
            self.stages[i+1].C_matrix = self.stages[i+1].C_matrix@L
    else:
        k = len(stages)
        print("k",k)
        for i in range(k-2,1,-1):
            print(i)
            Q, R = np.linalg.qr(np.hstack([stages[i].A_matrix,stages[i].B_matrix]).T,'reduced')

            Q = Q.T
            L = R.T
            print(L)

            stages[i].A_matrix=Q[:,:stages[i].A_matrix.shape[1]]
            stages[i].B_matrix=Q[:,stages[i].A_matrix.shape[1]:]
            stages[i-1].A_matrix = stages[i-1].A_matrix@L
            stages[i-1].C_matrix = stages[i-1].C_matrix@L
        return stages
        
    def make_observable(self,tol = 1e-8):
        """ make_observable makes the system observable
            This also includes a possible reduction of the state dimension

        Args:
            tol: (float, optional): used for the rank determination in the SVD
                (Note: this is not nececarraly the tolerance for the overall observability matrix)

        TODO: it is unclear if this is also true if the first/final state has dim =0
        """
        if self.causal:
            k = len(self.stages)
            for i in range(k-1, 0,-1):
                U,s,Vt= np.linalg.svd(np.vstack([self.stages[i].C_matrix,self.stages[i].A_matrix]))
                n = np.count_nonzero(s>tol)

                rs = np.sqrt(s)
                Us=U*rs
                sVt=Vt*rs.reshape(-1,1)

                self.stages[i].C_matrix=Us[self.stages[i].C_matrix.shape[0]:,:n]
                self.stages[i].A_matrix=Us[:self.stages[i].C_matrix.shape[0],:n]
                self.stage[i-1].A_matrix=sVt[:,:n]@self.stage[i-1].A_matrix
                self.stage[i-1].B_matrix=sVt[:,:n]@self.stage[i-1].B_matrix
        else:
            raise NotImplementedError("Not yet implemented")

    def make_minimal(self,tol = 1e-8):
        """ make_minimal makes the system minimal
            This removes unnececary dimension

        Args:
            tol: (float, optional): used for the rank determination in the SVD
                (Note: this is not nececarraly the tolerance for the overall
                observability and reachability matries)

        TODO: it is unclear if this is also true if the first/final state has dim =0
        """
        self.make_reachable(tol=tol)
        self.make_observable(tol=tol)

In [92]:
np.linalg.qr(np.random.rand(4,2),'reduced')

(array([[-0.60716048,  0.25938706],
        [-0.70913331, -0.55230783],
        [-0.19898132,  0.73545793],
        [-0.29814852,  0.29457776]]),
 array([[-1.24422543, -1.00430577],
        [ 0.        ,  0.95658796]]))

In [93]:
testsys_qr = testsys.copy()
make_input_normal_qr(testsys_qr)
testsys_qr.is_input_normal()

True

In [94]:
testsys_a_qr = testsys_a.copy()
make_input_normal_qr(testsys_a_qr)
testsys_qr.is_input_normal()

k 4
2
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 5.55111512e-17 -2.77555756e-17  1.00000000e+00  0.00000000e+00]
 [ 8.67361738e-18  2.77555756e-17  1.11022302e-16  1.00000000e+00]]


True

In [45]:
make_reachable(testsys_qr)
testsys.is_input_normal()

True

In [17]:
testsys.reachability_matrix(3)@testsys.reachability_matrix(3).T

array([[ 1.00000000e+00,  1.66533454e-16,  0.00000000e+00,
        -1.59594560e-16],
       [ 1.66533454e-16,  1.00000000e+00,  1.94289029e-16,
         5.55111512e-17],
       [ 0.00000000e+00,  1.94289029e-16,  1.00000000e+00,
         1.11022302e-16],
       [-1.59594560e-16,  5.55111512e-17,  1.11022302e-16,
         1.00000000e+00]])

In [18]:
system_b = system.causal_system.copy()
system_a = system.causal_system.copy()

In [19]:
system_a+system_b

TypeError: unsupported operand type(s) for +: 'StrictSystem' and 'StrictSystem'

In [ ]:
system.anticausal_system.observability_matricies()

In [ ]:
system.anticausal_system.reachability_matricies()

In [ ]:
system.causal_system.observability_matricies()

In [ ]:
system@system

In [ ]:
A = np.array([1,2])

In [ ]:
A.__matmul__?

In [ ]:
print(system)

In [ ]:
system_a.is_canonical()

In [ ]:
O = np.random.rand(5,5)
Uo,so,Vot = np.linalg.svd(O)
so = np.linspace()

R = np.random.rand(5,5)
Ur,sr,Vrt = np.linalg.svd(R)

In [ ]:
so = 10**(-np.linspace(1,25,5))
sr = 10**(-np.linspace(1,25,5))

In [ ]:
U,s,V = np.linalg.svd(np.diag(so)@Vot@Ur@np.diag(sr))

In [ ]:
plt.matshow(abs(U))

In [ ]:
so

In [ ]:
10**(-np.linspace(1,10,5))

In [ ]:
U

In [ ]:
d = np.diag(U)

In [ ]:
d

In [ ]:
np.fill_diagonal(U,0)
U

In [ ]:
np.allclose?

In [ ]:
np.alltrue?

In [ ]:
np.allclose(d,1)

# Test observability and reachability

In [ ]:
stages = system_a.stages
reachs = system_a.reachability_matricies()
reachs

In [ ]:
system_a.reachability_matrix(4)

In [ ]:
obss = system_a.observability_matricies()
obss

In [ ]:
system_a.observability_matrix(3)

In [ ]:
system_t = system_a.transpose()
reachs = system_t.reachability_matricies()
reachs

In [ ]:
system_t.reachability_matrix(1)

In [ ]:
obss = system_t.observability_matricies()
obss

In [ ]:
system_t.observability_matrix(1)

In [ ]:
system_causal = system_a

all_obs = []
all_reach = []
all_hankels = []
matrix_rec = system_causal.to_matrix()
print(matrix_rec)
i_in= 0
i_out = 0
for i in range(1,len(system_causal.stages)):
    print(i)
    all_obs.append(system_causal.observability_matrix(i))
    all_reach.append(system_causal.reachability_matrix(i))

    i_in += system_causal.dims_in[i-1]
    i_out += system_causal.dims_out[i-1]
    all_hankels.append(matrix_rec[i_out:,:i_in])

np.all([np.allclose(all_hankels[i],all_obs[i]@all_reach[i]) for i in range(len(all_hankels))])

system_anticausal = system_a.transpose()
all_obs = []
all_reach = []
all_hankels = []
matrix_rec = system_anticausal.to_matrix()
print(matrix_rec)
i_in= sum(system_causal.dims_in)#-dims_in[-1]
i_out = sum(system_causal.dims_out)#-dims_out[-1]
for i in range(len(system_anticausal.stages)-2,-1,-1):
    print(i)
    all_obs.append(system_anticausal.observability_matrix(i))
    all_reach.append(system_anticausal.reachability_matrix(i))

    i_in -= system_anticausal.dims_in[i+1]
    i_out -= system_anticausal.dims_out[i+1]
    all_hankels.append(matrix_rec[:i_out,i_in:])


np.all([np.allclose(all_hankels[i],all_obs[i]@all_reach[i]) for i in range(len(all_hankels))])

In [ ]:
system_anticausal.reachability_matrix(0)

In [ ]:
system_t.observability_matrix(1)@system_t.reachability_matrix(1)

In [ ]:
system_t.to_matrix()[:6,6:]

In [ ]:
k = len(stages)

i=0
mats = [stages[i-1].B_matrix]
As = stages[i-1].A_matrix
for l in range(i-2,-1,-1):
    mats.append(As@stages[l].B_matrix)
    As = As@stages[l].A_matrix
mats.reverse()
reach = np.hstack(mats)

mats = [stages[i].C_matrix]
As = stages[i].A_matrix
for l in range(i+1,k,1):
    mats.append(stages[l].C_matrix@As)
    As = stages[l].A_matrix@As
obs = np.vstack(mats)

In [ ]:
system_t.dims_in

In [ ]:
k = 3
for j in range(k-1,-1,-1):
    print(j)

# Test how to do operations

In [ ]:
class Expr:
    def __init__(self,a):
        self.a = a
        
    def __mul__(self,other):
        print("mult",self.a,other)
        return Expr(","+self.a +"*"+other.a+",")

    def __matmul__(self,other):
        print("matmult",self.a,other)
        return Expr(","+self.a +"@"+other.a+",")
    
    def __add__(self,other):
        print("add",self.a,other)
        return Expr(","+self.a +"+"+other.a+",")

    def __neg__(self):
        print("neg",self.a)
        return Expr(",-"+self.a+",")
    def __str__(self):
        return self.a
    def bla(self):
        print(self.a,"bla")
        return(self)

In [ ]:
-(Expr("1")+Expr("2") *Expr("3")).bla()

In [ ]:
Expr("1")*Expr("2")*Expr("3")

In [ ]:
Expr("1")@Expr("2")@Expr("3")

In [ ]:
Expr("1")@Expr("2")*Expr("3")

In [ ]:
Expr("1")*Expr("2")@Expr("3")

In [ ]:
v = np.linspace(1,0,10)
v[0]=1
v[1]=1
v

In [ ]:
np.all(v[1:]-v[:-1]<1e-16)

In [ ]:
v[1:]-v[:-1]